# Housing Price Prediction w/ Kaggle Dataset

### Purpose: To refine my data science skills even more and have more concrete evidence of them.

### Tools: Jupyter Notebook, Python 3.11, TensorFlow, Pandas

### Section 1: Data Exploration

#### We are already given a description of all of the data. We will load it here:

In [1]:
with open('HousingPredictionData/data_description.txt','r') as read_file:
    print(read_file.read())

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

#### Based on this document, we will have to do a lot of one-hot encoding for all of these categorical variables (i.e. catogires like excellent, good, ok, bad, very bad).

**We need to do one hot encoding for the following categories (according to the file):** MSSubClass, MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, RoofStyle, RoofMat1, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, CentralAir, MiscFeature, MoSold, SaleType, SaleCondition  
**Continuous variables that need to be normalized:** LotFrontage, LotArea, MasVnrArea, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageYrBlt, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal,YearBuilt, YearRemodAdd, YrSold

#### Note that the data descriptor file is wrong about two of the column names! Bedroom and Kitchen do not exist- the columns are called BedroomAbvGr and KitchenAbvGr, respectively. Figured this out later when got an error using the pipeline.

### Section 2: Loading and Transforming Data

#### Obviously, we have a huge number of features here. We need to apply one hot encoding to our categorical features and apply normalization to our continuous features. In order to make sure that our normalization stays consistent, we will use a OneHot model and train it using the data. 

#### We will now load our data into a Pandas DataFrame for processing. 

In [2]:
#### In a later code segment, pandas throws many warnings, but based on my research, these warnings are meaningless. We use a library to supress them
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

In [3]:
myDataFrame = pd.read_csv("HousingPredictionData/train.csv")

In [4]:
myDataFrame.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


#### We will remove SalePrice and call it the train_y value because it is the value that we are supposed to be predicting. We will divide the train_y by 100,000 to avoid exploding gradients and other issues.

In [5]:
train_y = myDataFrame.loc[:, myDataFrame.columns == 'SalePrice']
train_y = train_y/100000
myDataFrame = myDataFrame.loc[:, myDataFrame.columns != 'SalePrice']

#### We will first define which features we will onehot encode, and which features we will minmax (using our analysis above).

In [6]:
oneHotList=['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 
            'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
            'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
            'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional',
            'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'CentralAir',
            'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition']
minMaxList=['LotFrontage', 'LotArea', 'MasVnrArea','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea',
            'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr', 'TotRmsAbvGrd','Fireplaces','GarageYrBlt',
            'GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal', 'YearBuilt','YearRemodAdd','YrSold']  

#### We will now check to make sure that the data type for each column is the correct type.

In [7]:
myDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

#### MSSubClass should be of type string for our one hot encoding. We will make sure to cast all of the columns that we need to as the type that we expect.

In [8]:
myDataFrame[oneHotList]=myDataFrame[oneHotList].astype(str)

#### We will write a custom function in order to one hot encode our data. We will also apply normalization to the continuous data, but we will get into the details later.

In [9]:
myEncodingDict={}
for column in oneHotList:
    myEncodingDict[column]=[]
    for category in myDataFrame[column].unique():
        myEncodingDict[column].append(column+category)
    for newcolumn in myEncodingDict[column]:
        myDataFrame[newcolumn]=0;
        myDataFrame.loc[(myDataFrame[column] == newcolumn[len(column):]), newcolumn] = 1;

C:\Users\pillo\AppData\Local\Temp\ipykernel_9084\2210275607.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  myDataFrame[newcolumn]=0;
C:\Users\pillo\AppData\Local\Temp\ipykernel_9084\2210275607.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  myDataFrame[newcolumn]=0;
C:\Users\pillo\AppData\Local\Temp\ipykernel_9084\2210275607.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [10]:
myDataFrame[['SaleConditionNormal','SaleCondition']].head(10)

,SaleConditionNormal,SaleCondition
0,1,Normal
1,1,Normal
2,1,Normal
3,0,Abnorml
4,1,Normal
5,1,Normal
6,1,Normal
7,1,Normal
8,0,Abnorml
9,1,Normal


#### Seems like our one-hot encoder worked.

#### Now we have to make sure that we have no null values in our columns. We are ok with null values in our categorical variables - this will one-hot encode to its own column, and that will not affect the data processing. We are not ok with null values in our continuous variables - the nulls will cause errors. Thus, we will set the nulls to 0.

In [11]:
def fixNA(dataframe):
    dataframe[minMaxList]=dataframe[minMaxList].fillna(0)
    return dataframe

In [12]:
myDataFrame=fixNA(myDataFrame)

#### Now we have to normalize our data values. We will make another custom function for the continuous data in order to min max it. Note that we are keeping the values that we are using to one hot encode and we will be keeping the values that we use to minmax these variables

In [13]:
myMinMaxDict={}
for column in minMaxList:
    myMinMaxDict[column]=[myDataFrame[column].min(),myDataFrame[column].max()]
    myDataFrame[column]-=myMinMaxDict[column][0]
    myDataFrame[column]/=myMinMaxDict[column][1]
myDataFrame=myDataFrame.drop(columns=oneHotList)
myDataFrame

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleTypeCWD,SaleTypeConLw,SaleTypeCon,SaleTypeOth,SaleConditionNormal,SaleConditionAbnorml,SaleConditionPartial,SaleConditionAdjLand,SaleConditionAlloca,SaleConditionFamily
0,1,0.207668,0.033218,0.065174,0.026368,0.122500,706,0.000000,0.064212,0.140098,...,0,0,0,0,1,0,0,0,0,0
1,2,0.255591,0.038561,0.051741,0.012935,0.000000,978,0.000000,0.121575,0.206547,...,0,0,0,0,1,0,0,0,0,0
2,3,0.217252,0.046226,0.064179,0.025871,0.101250,486,0.000000,0.185788,0.150573,...,0,0,0,0,1,0,0,0,0,0
3,4,0.191693,0.038328,0.021393,0.009950,0.000000,216,0.000000,0.231164,0.123732,...,0,0,0,0,0,1,0,0,0,0
4,5,0.268371,0.060210,0.063682,0.024876,0.218750,655,0.000000,0.209760,0.187398,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,0.198083,0.030742,0.063184,0.024876,0.000000,0,0.000000,0.407962,0.155974,...,0,0,0,0,1,0,0,0,0,0
1456,1457,0.271565,0.055170,0.052736,0.018905,0.074375,790,0.110583,0.252140,0.252373,...,0,0,0,0,1,0,0,0,0,0
1457,1458,0.210863,0.035968,0.034328,0.027861,0.000000,275,0.000000,0.375428,0.188543,...,0,0,0,0,1,0,0,0,0,0
1458,1459,0.217252,0.039104,0.038806,0.022886,0.000000,49,0.698100,0.000000,0.176432,...,0,0,0,0,1,0,0,0,0,0


#### Great! Now we can remove the columns that are highly correlated in a process called feature reduction. This will let our model be easier to train and not have redundant inputs.

In [14]:
import numpy as np

correlationMatrix = myDataFrame.corr().abs().replace(1.0,0)
unstacked = correlationMatrix.unstack()
sortedCorrelations = unstacked.sort_values(kind="quicksort")
sortedCorrelations.tail(50)

LandSlopeMod          LandSlopeGtl            0.908607
LandSlopeGtl          LandSlopeMod            0.908607
HouseStyle1.5Unf      MSSubClass45            0.925181
MSSubClass45          HouseStyle1.5Unf        0.925181
LotShapeIR1           LotShapeReg             0.925959
LotShapeReg           LotShapeIR1             0.925959
RoofStyleHip          RoofStyleGable          0.933462
RoofStyleGable        RoofStyleHip            0.933462
HouseStyle1.5Fin      MSSubClass50            0.940871
MSSubClass50          HouseStyle1.5Fin        0.940871
HouseStyleSLvl        MSSubClass80            0.942259
MSSubClass80          HouseStyleSLvl          0.942259
MiscFeaturenan        MiscFeatureShed         0.950890
MiscFeatureShed       MiscFeaturenan          0.950890
BsmtExposurenan       BsmtFinType2nan         0.972981
BsmtFinType2nan       BsmtExposurenan         0.972981
Exterior2ndMetalSd    Exterior1stMetalSd      0.973065
Exterior1stMetalSd    Exterior2ndMetalSd      0.973065
Exterior2n

#### We will drop the following columns from the final result: StreetPave, GarageTypenan, and PoolQCnan, because of their high correlation to other columns. We also will drop Id because it should not be used for prediction.

In [15]:
dropList=['StreetPave','GarageTypenan','PoolQCnan','Id']
myDataFrame=myDataFrame.drop(columns=dropList)

#### Now we are ready to build a model! Wahoo! Our data is prepared. The following code segment is a summary of what we have done to the data and is a pipeline so that we can manipulate any sort of test data given.

In [16]:
def myPipeline(dataLocation,myEncodingDict,myMinMaxDict,dropList):
    myPipeDataFrame = pd.read_csv(dataLocation)
    myPipeDataFrame = myPipeDataFrame.loc[:, myPipeDataFrame.columns != 'SalePrice']
    myPipeDataFrame[oneHotList]=myPipeDataFrame[oneHotList].astype(str)
    for column in oneHotList:
        for newcolumn in myEncodingDict[column]:
            myPipeDataFrame[newcolumn]=0;
            myPipeDataFrame.loc[(myPipeDataFrame[column] == newcolumn[len(column):]), newcolumn] = 1;
    myPipeDataFrame=fixNA(myPipeDataFrame)
    for column in minMaxList:
        myPipeDataFrame[column]-=myMinMaxDict[column][0]
        myPipeDataFrame[column]/=myMinMaxDict[column][1]
    myPipeDataFrame=myPipeDataFrame.drop(columns=oneHotList)
    myPipeDataFrame=myPipeDataFrame.drop(columns=dropList)
    return myPipeDataFrame

## Section 3: Building and Training the Model

#### We will now build the model using tensorflow and keras.

In [17]:
#We will use the layers package to construct our model
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras import layers

In [18]:
#definition of model
model=Sequential()
model.add(layers.Dense(500, activation="sigmoid"))
model.add(layers.Dense(1, activation="leaky_relu"))
#complies using the adam optimizer and mse loss function (loss used to measure how well the function is training)
optimizer=Adam(learning_rate=.00002)
model.compile(optimizer=optimizer, loss='mse',metrics=['mean_absolute_error'])

#### Now we will train the model using the dataframe that we have cleaned.

In [19]:
model.fit(tf.convert_to_tensor(myDataFrame),tf.convert_to_tensor(train_y),epochs=3500,verbose=True,batch_size=1500)

Epoch 1/3500
1/1 [==============================] - 0s 329ms/step - loss: 2.6768 - mean_absolute_error: 1.4151
Epoch 2/3500
1/1 [==============================] - 0s 8ms/step - loss: 2.6583 - mean_absolute_error: 1.4082
Epoch 3/3500
1/1 [==============================] - 0s 8ms/step - loss: 2.6400 - mean_absolute_error: 1.4013
Epoch 4/3500
1/1 [==============================] - 0s 5ms/step - loss: 2.6217 - mean_absolute_error: 1.3944
Epoch 5/3500
1/1 [==============================] - 0s 19ms/step - loss: 2.6035 - mean_absolute_error: 1.3874
Epoch 6/3500
1/1 [==============================] - 0s 8ms/step - loss: 2.5854 - mean_absolute_error: 1.3804
Epoch 7/3500
1/1 [==============================] - 0s 8ms/step - loss: 2.5673 - mean_absolute_error: 1.3735
Epoch 8/3500
1/1 [==============================] - 0s 5ms/step - loss: 2.5494 - mean_absolute_error: 1.3665
Epoch 9/3500
1/1 [==============================] - 0s 16ms/step - loss: 2.5315 - mean_absolute_error: 1.3595
Epoch 10/3500
1

#### Let's predict some house prices using test data.

In [20]:
myResult=model.predict(tf.convert_to_tensor(myPipeline('HousingPredictionData/test.csv',myEncodingDict,myMinMaxDict,dropList)))

C:\Users\pillo\AppData\Local\Temp\ipykernel_9084\2405438221.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  myPipeDataFrame[newcolumn]=0;
C:\Users\pillo\AppData\Local\Temp\ipykernel_9084\2405438221.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  myPipeDataFrame[newcolumn]=0;
C:\Users\pillo\AppData\Local\Temp\ipykernel_9084\2405438221.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

46/46 [==============================] - 0s 1ms/step


#### Let's check the predictions:

In [21]:
#rescales the result
myResult=myResult*100000
myResult[:10]

array([[117847.52607346],
       [183440.05346298],
       [211841.39251709],
       [219995.73707581],
       [184496.52194977],
       [170443.57061386],
       [179492.00868607],
       [157352.90050507],
       [172431.2543869 ],
       [142981.32658005]])

#### Great! This looks like some reasonable predictions. Now we will export these predictions as a csv file in the required format (specified by competition).

In [22]:
#Used to load ids for the final csv file
idDataFrame=pd.read_csv('HousingPredictionData/test.csv')
idDataFrame=idDataFrame['Id']

In [24]:
import csv
with open('HousingPredictionData/submission.csv','w') as file:
    writer=csv.writer(file)
    writer.writerow(['Id','SalePrice'])
    counter=0
    for prediction in myResult:
        writer.writerow([str(idDataFrame[counter]),(prediction[0])])
        counter+=1
    

#### With this notebook, I learned a lot more about data cleaning and preparation. I realise that my skills are lacking in the model building department. In my next project, I hopoe to hone these skills.